Подключим библиотеки и google drive

In [ ]:
from google.colab import drive
from os import listdir, getcwd

import tensorflow as tf
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization, Activation, MaxPooling2D, \
    concatenate, Embedding, multiply, ReLU, Concatenate, ZeroPadding2D  # Базовые слои keras
from keras.layers.advanced_activations import LeakyReLU  # LeakyReLU - функция активации
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose  # Сверточные слои keras
from keras.models import Sequential, Model  # Формат модели keras
from keras.optimizers import adam_v2
from keras.utils.vis_utils import plot_model
from keras.backend import set_image_data_format
from keras.losses import Loss
from keras.regularizers import L2, L1
from tensorflow import random_normal_initializer, reshape, reduce_mean, math, subtract

import matplotlib.pyplot as plt  # Отображение графиков
import matplotlib.ticker as ticker

import numpy as np  # Numpy массивы
from keras_preprocessing.image import load_img, img_to_array
from numpy import asarray, log10
from sklearn.utils import shuffle



In [ ]:
drive.mount('/content/drive')
#!mkdir -p GAN/images/ # Создаем каталог 'images'
#%cd /content/drive/My Drive/Colab_Notebooks/DiplomaWork
%cd /content/drive/MyDrive/DiplomaWork/
print(listdir())

img_size_h = img_size_w = 128
savedImgDir = 'Generated_Imgs/EpochProgress'
generateMapTo = 'Generated_Imgs'
#dataset = 'datasets/256x256_no_bias'
#dataset = 'datasets/allInGeneral_bias_128x128'
dataset = 'datasets/mountains_128x128'
checkpointDir = 'gan_weights/first'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1q4AUodhojBvTWVdTGRefmvHxA2bbifah/DiplomaWork
['DiplomaWork.ipynb', 'datasets', 'Convolutional_disc', 'Convolutional_disc_gen', 'tendence.png', 'Generated_Imgs', 'pix2pix.ipynb', 'gan_weights', 'Copy of Fedorov_A GAN.ipynb', 'dcgan.ipynb', 'acc_and_loss.png', 'schemes.drawio', 'discriminator_conv_arch.png', 'GAN_arch.png', 'generator_conv_arch.png', 'gan_model_plot.png', 'Pix2PixTry.ipynb', 'DiplomaWork (U-net working).ipynb']


Определим структуру GAN

In [ ]:
def load_data(path):
    x_train = list()
    temp_x = load_images_from_dir(path + '/mountains')
    x_train.extend(temp_x)

    return (asarray(x_train))


# load all images in a directory into memory
def load_images_from_dir(path):
    pic_list = list()
    # enumerate filenames in directory, assume all are images
    for filename in listdir(path):
        # load and resize the image
        pixels = load_img(path + '/' + filename, color_mode='grayscale')
        # convert to numpy array
        pixels = img_to_array(pixels)
        pixels = (pixels.astype(np.float32) - 127.5) / 127.5
        pic_list.append(pixels)

    
    return asarray(pic_list)

class GeneratorLoss(Loss):
  def __init__(self):
    super().__init__()
  def call(self, y_true, y_pred):    
     return -reduce_mean(math.log(abs(y_true-y_pred)))

class DiscriminatorLoss(Loss):
  def __init__(self):
    super().__init__()
  def call(self, y_true, y_pred):     
    return -reduce_mean(math.log(1 - abs(y_true-y_pred )))                                                    #abs(y_true-y_pred): nnot guess all = 1 guess all = 0
    #return -reduce_mean(math.log(1 - abs(y_true-y_pred) - 1e-8))
    #return -reduce_mean(math.log(1.-y_pred)) -reduce_mean(math.log(y_true))
    

class GAN():
    def __init__(self):
        self.img_rows = img_size_h 
        self.img_cols = img_size_w 
        self.channels = 1 
        self.latent_dim = img_size_h*img_size_w*self.channels  
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.noise_shape = (8, 8, 100)
        self.patch_shape = (16, 16, 1)

        self.L2_coef = 1e-5

        self.cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
        self.generator_optimizer = tf.keras.optimizers.Adam(1e-4)
        self.discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

        #self.generator_optimizer = adam_v2.Adam(learning_rate=0.0005, beta_1=0.5)
        #self.discriminator_optimizer = adam_v2.Adam(learning_rate=0.0005, beta_1=0.5)

        #self.generator = self.make_generator_model()
        #self.discriminator = self.make_discriminator_model()

        self.generator = self.make_generator_false()
        #self.discriminator = self.make_discriminator_false()

        #self.generator = self.build_conv_generator()
        self.discriminator = self.build_conv_discriminator_false()


        self.checkpoint = tf.train.Checkpoint(#generator_optimizer=self.generator_optimizer,
                                              #discriminator_optimizer=self.discriminator_optimizer,
                                              generator=self.generator,
                                              discriminator=self.discriminator)
        
        self.build_GAN()

    def build_GAN(self):
        initial_learning_rate = 0.001
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate,
            decay_steps=100,
            decay_rate=0.975,
            staircase=True)
        
        optimizer = adam_v2.Adam(learning_rate=lr_schedule, beta_1=0.5) #lr = lr_schedule

        self.discriminator.compile(loss='binary_crossentropy', 
                                   #loss=DiscriminatorLoss(),
                                   optimizer=self.discriminator_optimizer,
                                   metrics=['accuracy']
                                   )

        self.generator.compile(loss='binary_crossentropy', 
                               #loss=GeneratorLoss(),
                               optimizer=self.generator_optimizer
                               )

        z = Input(shape=self.noise_shape, name="GAN_In_build") 
        img = self.generator(z)

        self.discriminator.trainable = False 
        valid = self.discriminator(img)

        self.combined = Model(z, valid, name="GAN") 
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        #self.combined.compile(loss = GeneratorLoss(), optimizer=optimizer)
        self.combined.summary()
        plot_model(self.combined, to_file='GAN_arch.png', show_shapes=True, show_layer_names=True)
 
    def downsample(self, filters, size, apply_batchnorm = False):
        initializer = random_normal_initializer(0., 0.02)

        result = Sequential()
        result.add(Conv2D(filters, size, strides=2, padding='same',
                          kernel_initializer=initializer, 
                          kernel_regularizer=L2(self.L2_coef),
                          use_bias=False))

        result.add(LeakyReLU())        
        
        if apply_batchnorm:
            result.add(BatchNormalization())

        return result

    def upsample(self, filters, size, apply_dropout=False):
        initializer = random_normal_initializer(0., 0.02)

        result = Sequential()
        result.add(
            Conv2DTranspose(filters, size, strides=2,     #Conv2DTranspose
                            padding='same',
                            kernel_initializer=initializer,
                            kernel_regularizer=L2(self.L2_coef),
                            use_bias=False))
        
        result.add(ReLU())
        result.add(BatchNormalization())

        if apply_dropout:
            result.add(Dropout(0.5))


        #result.add(LeakyReLU(alpha=0.2))

        return result

    def build_conv_discriminator(self):  # Функция создания сверточного дискрминатора
        model = Sequential()

        model.add(Dense(1, input_shape=self.img_shape, name="Discriminator_In"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(128, (5, 5), strides=1,
                         padding="same"))  # Создаем слой  Conv2D (размерность входных данных (img_shape), ядро свертки = 4, окно свертки = (3,3))
        model.add(LeakyReLU(alpha=0.2))  # Добавляем слой активационной функции с параметром 0.2
        model.add(Conv2D(128, (5, 5), strides=1,
                         padding="same"))  # Создаем слой  Conv2D (размерность входных данных (img_shape), ядро свертки = 4, окно свертки = (3,3))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(128, (5, 5), strides=1,
                         padding="same"))  # Создаем слой  Conv2D (размерность входных данных (img_shape), ядро свертки = 4, окно свертки = (3,3))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Flatten())  # Добавляем слой Flatten ()
        #model.add(Dense(32))  # Добавляем Dense-слой на 512 нейронов
        #model.add(LeakyReLU(alpha=0.2))  # Добавляем слой активационной функции с параметром 0.2
        model.add(Dense(1, activation='sigmoid',
                        name="Discriminator_Out"))  # Добавляем Dense-слой c 1 нейроном с активационной функцией sigmoid, поскольку нам нужно категорировать входноые изображения на два класса 1 - из тестовой выборки и 0 - сформирован генератором.

        model.summary()
        plot_model(model, to_file='discriminator_conv_arch.png', show_shapes=True, show_layer_names=True)
        
        #plot_model(discriminator_conv, to_file='discriminator_conv_arch1.png', show_shapes=True, show_layer_names=True)

        #return discriminator_conv
        return model


    def build_conv_discriminator_false(self):  # Функция создания сверточного дискрминатора
        inputs = Input(shape=self.img_shape, name="Discriminator_In")

        down_stack = [            
            #self.downsample(64, 5, apply_batchnorm = True),  # (batch_size, 64, 64, 64)
            self.downsample(128, 5, apply_batchnorm = True),  # (batch_size, 32, 32, 128)
            self.downsample(256, 5, apply_batchnorm = True),  # (batch_size, 64, 64, 64)
            self.downsample(512, 5, apply_batchnorm = True),  # (batch_size, 64, 64, 64)
            #self.downsample(1024, 5, apply_batchnorm = True),  # (batch_size, 16, 16, 256)
        ]


        initializer = random_normal_initializer(0., 0.02)
        last = Conv2D(1, 1, strides=1,
                                              padding='same',
                                             kernel_initializer=initializer,
                                            activity_regularizer=L2(self.L2_coef),
                                            activation='sigmoid')  # (batch_size, 128, 128, 1))


        flat = Flatten()
        #initializer = random_normal_initializer(0., 0.02)
        lastlast = Dense(1, activation='sigmoid',
                         name="Discriminator_Out")                                               
        x = inputs

        # Downsampling through the model
        for down in down_stack:
            x = down(x)

        x = last(x)
        #x = flat(x)
        #x = lastlast(x)

        model = Model(inputs=inputs, outputs=x)

        model.summary()
        plot_model(model, to_file='discriminator_conv_arch.png', show_shapes=True, show_layer_names=True)
        
        #plot_model(discriminator_conv, to_file='discriminator_conv_arch1.png', show_shapes=True, show_layer_names=True)

        #return discriminator_conv
        return model


    def make_discriminator_model(self):
        model = Sequential()

        #model.add(Conv2D(64, (5,5), strides=(2, 2), padding='same', input_shape = self.img_shape, kernel_regularizer=L2(self.L2_coef)))
        #model.add(LeakyReLU())
        #model.add(BatchNormalization(momentum = 0.5))


        model.add(Conv2D(128, (5,5), strides=(2, 2), padding='same', activation='tanh', input_shape = self.img_shape, kernel_regularizer=L2(self.L2_coef)))
        #model.add(LeakyReLU())
        model.add(BatchNormalization())


        model.add(Conv2D(256, (5,5), strides=(2, 2), padding='same', activation='tanh', kernel_regularizer=L2(self.L2_coef)))
        #model.add(LeakyReLU())
        model.add(BatchNormalization())

        #model.add(Conv2D(512, (5, 5), strides=(1, 1), padding='same', activity_regularizer=L2(self.L2_coef)))
        #model.add(LeakyReLU())
        #model.add(Dropout(0.3))
        
        model.add(Conv2D(512, (5,5), strides=(2, 2), padding='same', activation='tanh', kernel_regularizer=L2(self.L2_coef)))
        #model.add(LeakyReLU())
        model.add(BatchNormalization())
                

        model.add(Conv2D(1024, (5,5), strides=(2, 2), padding='same', activation='tanh', kernel_regularizer=L2(self.L2_coef)))
        #model.add(LeakyReLU())
        model.add(BatchNormalization())

        model.add(Conv2D(1, (5,5), strides=(1, 1), padding='same', activation='tanh', kernel_regularizer=L2(self.L2_coef)))
        #model.add(LeakyReLU())
        model.add(BatchNormalization())
        

        #model.add(Conv2D(1, (3,3), strides=(1, 1), padding='same', use_bias=False))
        #model.add(LeakyReLU())
        #model.add(BatchNormalization(momentum = 0.5))

        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))


        model.summary()
        plot_model(model, to_file='discriminator_conv_arch.png', show_shapes=True, show_layer_names=True)
        return model

    def make_discriminator_false(self):  # Функция создания сверточного дискрминатора
        inputs = Input(shape=self.img_shape, name="Discriminator_In")

        down_stack = [            
            #self.downsample(64, 5, apply_batchnorm = False),  # (batch_size, 64, 64, 64)
            #self.downsample(128, 3, apply_batchnorm = True),  # (batch_size, 32, 32, 128)
            self.downsample(256, 3, apply_batchnorm = True),  # (batch_size, 64, 64, 64)
            self.downsample(512, 3, apply_batchnorm = True),  # (batch_size, 64, 64, 64)
            self.downsample(1024, 3, apply_batchnorm = True),  # (batch_size, 16, 16, 256)
        ]


        #initializer = random_normal_initializer(0., 0.02)
        #last = Conv2D(32, 5, strides=1,
        #                                      padding='same',
        #                                     kernel_initializer=initializer,
        #                                    activity_regularizer=L2(0.01))
                                          #activation='tanh  # (batch_size, 128, 128, 1)


        flat = Flatten()
        #initializer = random_normal_initializer(0., 0.02)
        lastlast = Dense(1, activation='sigmoid',
                         name="Discriminator_Out")                                               
        x = inputs

        # Downsampling through the model
        for down in down_stack:
            x = down(x)

        #x = last(x)
        x = flat(x)
        x = lastlast(x)

        model = Model(inputs=inputs, outputs=x)

        model.summary()
        plot_model(model, to_file='discriminator_conv_arch.png', show_shapes=True, show_layer_names=True)
        
        #plot_model(discriminator_conv, to_file='discriminator_conv_arch1.png', show_shapes=True, show_layer_names=True)

        #return discriminator_conv
        return model

    def make_generator_model(self):
        model = Sequential()
     
        model.add(Conv2DTranspose(1024, (5, 5), strides=(2, 2), padding='same', use_bias=True, activation='tanh', input_shape=self.noise_shape, kernel_regularizer=L2(self.L2_coef)))
        #model.add(BatchNormalization())
        #model.add(Dropout(0.5))

        #model.add(Conv2DTranspose(1024, (7, 7), strides=(1, 1), padding='same', use_bias=False, activation='relu', input_shape=self.noise_shape, kernel_regularizer=L2(self.L2_coef)))
        #model.add(BatchNormalization())
        #model.add(Dropout(0.5))

        model.add(Conv2DTranspose(512, (5, 5), strides=(2, 2), padding='same', use_bias=True, activation='tanh', kernel_regularizer=L2(self.L2_coef)))
        #model.add(BatchNormalization())
        #model.add(Dropout(0.5))

        #model.add(Conv2DTranspose(512, (7, 7), strides=(1, 1), padding='same', use_bias=False, activation='relu', input_shape=self.noise_shape, kernel_regularizer=L2(self.L2_coef)))
        #model.add(BatchNormalization())
        #model.add(Dropout(0.5))

        model.add(Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=True, activation='tanh', kernel_regularizer=L2(self.L2_coef)))
        #model.add(BatchNormalization())
        #model.add(Dropout(0.5))

        #model.add(Conv2DTranspose(256, (7, 7), strides=(1, 1), padding='same', use_bias=False, activation='relu', input_shape=self.noise_shape, kernel_regularizer=L2(self.L2_coef)))
        #model.add(BatchNormalization())
        #model.add(Dropout(0.5))

        model.add(Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=True, activation='tanh', kernel_regularizer=L2(self.L2_coef)))
        #model.add(BatchNormalization())
        #model.add(Dropout(0.5))

        #model.add(Conv2DTranspose(512, (7, 7), strides=(1, 1), padding='same', use_bias=False, activation='relu', input_shape=self.noise_shape, kernel_regularizer=L2(self.L2_coef)))
        #model.add(BatchNormalization())
        #model.add(Dropout(0.5))

        #model.add(Conv2DTranspose(64, (7, 7), strides=(2, 2), padding='same', use_bias=False, activation='relu', kernel_regularizer=L2(self.L2_coef)))
        #model.add(BatchNormalization(momentum = 0.5))

        #model.add(Conv2DTranspose(64, (7, 7), strides=(1, 1), padding='same', use_bias=False, activation='relu', input_shape=self.noise_shape, kernel_regularizer=L2(self.L2_coef)))
        #model.add(BatchNormalization(momentum = 0.5))
        #model.add(Dropout(0.5))

        #model.add(Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='relu', kernel_regularizer=L2(self.L2_coef)))
        #model.add(BatchNormalization(momentum = 0.5))
     
        model.add(Conv2DTranspose(1, (5, 5), strides=(1, 1), padding='same', activation='tanh', use_bias=True, kernel_regularizer=L2(self.L2_coef) ))

        model.summary()
        plot_model(model, to_file='generator_conv_arch.png', show_shapes=True, show_layer_names=True)  
        return model

    def make_generator_false(self):
        inputs = Input(shape=self.noise_shape, name="GAN_In_generator")

        up_stack = [
            self.upsample(1024, 5, apply_dropout = False),
            self.upsample(512, 5, apply_dropout = False),  # (batch_size, 2, 2, 1024)
            self.upsample(256, 5, apply_dropout = False),  # (batch_size, 4, 4, 1024)
            self.upsample(128, 5, apply_dropout = False),  # (batch_size, 8, 8, 1024)
            #self.upsample(64, 5),                        # (batch_size, 16, 16, 1024)
            #self.upsample(128, 5),                        # (batch_size, 32, 32, 256)
            #self.upsample(64, 5),                         # (batch_size, 64, 64, 64)
        ]

        initializer = random_normal_initializer(0., 0.02)
        #last = Conv2DTranspose(32, 3, strides=2,
        #                                       padding='same',
        #                                       kernel_initializer=initializer,
        #                                        kernel_regularizer=L2(self.L2_coef))
                                               #activation='tanh  # (batch_size, 128, 128, 1)

        lastlast = Conv2DTranspose(1, 5, strides=1,
                                               padding='same',
                                               kernel_initializer=initializer,
                                               activation='tanh',
                                              kernel_regularizer=L2(self.L2_coef),
                                               name = 'lastlast')  # (batch_size, 128, 128, 1)

        x = inputs

        # Upsampling and establishing the skip connections
        for up in up_stack:
            x = up(x)

        #x = last(x)
        x = lastlast(x)

        model = Model(inputs=inputs, outputs=x)

        model.summary()

        plot_model(model, to_file='generator_conv_arch.png', show_shapes=True, show_layer_names=True)  

        return model

    def build_conv_generator(self):
        inputs = Input(shape=self.img_shape, name="GAN_In_generator")

        down_stack = [
            self.downsample(64, 5, apply_batchnorm = False),  # (batch_size, 64, 64, 64)
            self.downsample(128, 5),  # (batch_size, 32, 32, 128)
            self.downsample(256, 5),  # (batch_size, 16, 16, 256)
            self.downsample(512, 5),  # (batch_size, 8, 8, 512)
            self.downsample(512, 5),  # (batch_size, 4, 4, 512)
            self.downsample(512, 5),  # (batch_size, 2, 2, 512)
            self.downsample(512, 5),  # (batch_size, 1, 1, 512)
        ]

        up_stack = [
            self.upsample(512, 5, apply_dropout = True),  # (batch_size, 2, 2, 1024)
            self.upsample(512, 5, apply_dropout = True),  # (batch_size, 4, 4, 1024)
            self.upsample(512, 5, apply_dropout = True),  # (batch_size, 8, 8, 1024)
            self.upsample(256, 5),                        # (batch_size, 16, 16, 1024)
            self.upsample(128, 5),                        # (batch_size, 32, 32, 256)
            self.upsample(64, 5),                         # (batch_size, 64, 64, 64)
        ]

        initializer = random_normal_initializer(0., 0.02)
        last = Conv2DTranspose(32, 3, strides=2,
                                               padding='same',
                                               kernel_initializer=initializer,
                                                kernel_regularizer=L2(self.L2_coef))
                                               #activation='tanh  # (batch_size, 128, 128, 1)

        lastlast = Conv2DTranspose(1, 3, strides=1,
                                               padding='same',
                                               kernel_initializer=initializer,
                                               activation='tanh',
                                              kernel_regularizer=L2(self.L2_coef),
                                               name = 'lastlast')  # (batch_size, 128, 128, 1)

        x = inputs

        # Downsampling through the model
        skips = []
        for down in down_stack:
            x = down(x)
            skips.append(x)

        #skips.pop()
        skips = reversed(skips[:-1])#-1
        

        # Upsampling and establishing the skip connections
        for up, skip in zip(up_stack, skips):
            x = up(x)
            x = Concatenate()([x, skip])

        x = last(x)
        x = lastlast(x)

        model = Model(inputs=inputs, outputs=x)

        model.summary()

        plot_model(model, to_file='generator_conv_arch.png', show_shapes=True, show_layer_names=True)  

        return model


    def build_conv_generator_false(self):
        inputs = Input(shape=self.noise_shape, name="GAN_In_generator")

        up_stack = [
            self.upsample(1024, 7, apply_dropout = True),  # (batch_size, 2, 2, 1024)
            self.upsample(512, 7, apply_dropout = True),  # (batch_size, 2, 2, 1024)
            self.upsample(256, 7, apply_dropout = True),  # (batch_size, 2, 2, 1024)
            self.upsample(128, 7, apply_dropout = True),  # (batch_size, 2, 2, 1024)
            #self.upsample(64, 7, apply_dropout = True)
        ]

        initializer = random_normal_initializer(0., 0.02)
        last = Conv2DTranspose(1, 7, strides=1,
                                               padding='same',
                                               kernel_initializer=initializer,
                                               activation='tanh')  # (batch_size, 128, 128, 1)

        '''initializer = random_normal_initializer(0., 0.02)
        last = Conv2DTranspose(32, 3, strides=1,
                                               padding='same',
                                               kernel_initializer=initializer,
                                               activity_regularizer=L2(0.01),
                                               name = 'last')  # (batch_size, 128, 128, 1)

        initializer = random_normal_initializer(0., 0.02)
        lastlast = Conv2DTranspose(1, 5, strides=1,
                                               padding='same',
                                               kernel_initializer=initializer,
                                               activity_regularizer=L2(0.01),
                                               activation='tanh',
                                               name = 'lastlast')  # (batch_size, 128, 128, 1)'''
        x = inputs

        # Downsampling through the model
        

        # Upsampling and establishing the skip connections
        for up in up_stack:
            x = up(x)

        x = last(x)
        #x = lastlast(x)

        model = Model(inputs=inputs, outputs=x)

        model.summary()

        plot_model(model, to_file='generator_conv_arch.png', show_shapes=True, show_layer_names=True)  

        return model


    def generator_loss(self, fake_output):
        #return -reduce_mean(math.log(1-fake_output))
        return self.cross_entropy(tf.ones_like(fake_output), fake_output)



    def discriminator_loss(self, real_output, fake_output):
        #return -reduce_mean(math.log(fake_output))   -reduce_mean(math.log(1-real_output))
        real_loss = self.cross_entropy(tf.ones_like(real_output), real_output)
        fake_loss = self.cross_entropy(tf.zeros_like(fake_output), fake_output)
        total_loss = real_loss + fake_loss
        return total_loss


  # Notice the use of `tf.function`
    # This annotation causes the function to be "compiled".
    @tf.function
    def train_step(self, images, batch_size):
        noise = np.random.normal(0, 1, (batch_size, self.noise_shape[0], self.noise_shape[1], self.noise_shape[2]))
        #noise = np.random.normal(0, 1, (batch_size, self.img_shape[0], self.img_shape[1], self.img_shape[2]))

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
          generated_images = self.generator(noise, training=True)

          real_output = self.discriminator(images, training=True)
          fake_output = self.discriminator(generated_images, training=True)

          gen_loss = self.generator_loss(fake_output)
          disc_loss = self.discriminator_loss(real_output, fake_output)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))

        return disc_loss, reduce_mean(fake_output),  gen_loss 

    def train(self, epochs, batch_size=128, save_interval=1000, save_images=False, tendention_С=None):
        '''
        X_train = load_data(dataset)
        X_train = shuffle(X_train)

        valid = np.ones((batch_size, 1))  # Создаем массив единиц длинной batch_size
        fake = np.zeros((batch_size, 1))  # Создаем массив нулей длинной batch_size

        # tendention_plot, axs, tendention_R, tendention_C = self.create_tendention_plot(save_interval, epochs)
        curr_colomn = 0
        d_loss_list = []
        g_loss_list = []
        acc_list = []
        loss_acc_plt_step = 5
        step_summ_g_loss = 0
        step_summ_d_loss = 0
        step_summ_acc = 0
        

        for epoch in range(epochs):
          idx = np.random.randint(0, X_train.shape[0], batch_size)  # Выбираем случайным образом batch_size картинок из исходной обучающей выбрки для тренировки дискриминатора
          imgs = X_train[idx]  # В переменную imgs записываем значение из X_train с индексами из idx

          disc_loss, d_acc, gen_loss  = self.train_step(imgs, batch_size)

          print("%d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (
            epoch, disc_loss, d_acc*100, gen_loss))  # Plot the progress

          if (epoch % save_interval == 0) or (
                epoch == epochs - 1):  # Выводим/сохраняем изображения каждые save_interval эпох и в конце цикла
            self.save_imgs(epoch, save_images)

        return


        '''
        X_train = load_data(dataset)
        print(X_train.shape[0])
        X_train = shuffle(X_train)

        #valid = np.ones((batch_size, 1)) 
        valid = np.ones((batch_size, self.patch_shape[0], self.patch_shape[1], self.patch_shape[2]))
        fake = np.zeros((batch_size, self.patch_shape[0], self.patch_shape[1], self.patch_shape[2]))
        #fake = np.zeros((batch_size, 1)) 

        # tendention_plot, axs, tendention_R, tendention_C = self.create_tendention_plot(save_interval, epochs)
        curr_colomn = 0
        d_loss_list = []
        g_loss_list = []
        acc_list = []
        loss_acc_plt_step = 5
        step_summ_g_loss = 0
        step_summ_d_loss = 0
        step_summ_acc = 0
        

        for epoch in range(epochs):
            # ---Train Discriminator---
            idx = np.random.randint(0, X_train.shape[0], batch_size)  # Выбираем случайным образом batch_size картинок из исходной обучающей выбрки для тренировки дискриминатора
            imgs = X_train[idx]  # В переменную imgs записываем значение из X_train с индексами из idx

            gen_imgs = []
            #noise_img = np.random.normal(0, 1, (batch_size, self.img_shape[0], self.img_shape[1], self.img_shape[2]))  # Формируем массив размерностью (batch_size, self.latent_dim) из нормально распределенных значений
            noise_img = np.random.normal(0, 1, (batch_size, self.noise_shape[0], self.noise_shape[1], self.noise_shape[2]))

            gen_imgs = self.generator.predict(noise_img)

            self.discriminator.trainable = True
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)  
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)                                            
            d_loss = np.add(d_loss_real, d_loss_fake) / 2  

            # --- Train Generator ---
            #noise = np.random.normal(0, 1, (batch_size, self.img_shape[0], self.img_shape[1], self.img_shape[2]))
            noise = np.random.normal(0, 1, (batch_size, self.noise_shape[0], self.noise_shape[1], self.noise_shape[2]))
            self.discriminator.trainable = False
            g_loss = self.combined.train_on_batch(x=noise, y=valid)  #combined

            step_summ_g_loss += d_loss[0]
            step_summ_d_loss += g_loss
            step_summ_acc += 100 * d_loss[1]

            if (epoch % loss_acc_plt_step == 0):
                acc_list = np.append(acc_list, step_summ_acc / loss_acc_plt_step)
                d_loss_list = np.append(d_loss_list, step_summ_d_loss / loss_acc_plt_step)
                g_loss_list = np.append(g_loss_list, step_summ_g_loss / loss_acc_plt_step)
                step_summ_g_loss = 0
                step_summ_d_loss = 0
                step_summ_acc = 0

            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (
            epoch, d_loss[0], 100 * d_loss[1], g_loss))  # Plot the progress

            if( math.is_nan(g_loss) or math.is_inf(g_loss)):
              break

            if (epoch % save_interval == 0) or (
                    epoch == epochs - 1) :  # Выводим/сохраняем изображения каждые save_interval эпох и в конце цикла
                self.save_imgs(epoch, save_images)
                # self.save_tendention(tendention_plot, axs, tendention_C, curr_colomn)
                curr_colomn += 1
            # self.close_tendention(tendention_plot)

            if (epoch % 5000 == 0) or (
                    epoch == epochs - 1) :  # Выводим/сохраняем изображения каждые save_interval эпох и в конце цикла
                self.checkpoint.save(file_prefix = checkpointDir)

        x_lin = np.linspace(0, epochs - 1, epochs // loss_acc_plt_step)
        fig, axs = plt.subplots(2, figsize=(300, 10), dpi=100)

        # fig.subplots_adjust(bottom=spacing)
        axs[0].plot(x_lin, log10(d_loss_list), linewidth=1, color='b', label='d_loss')
        axs[0].plot(x_lin, log10(g_loss_list), linewidth=1, color='r', label='g_loss')
        axs[0].set(ylabel='loss_value', xlabel='epoch')
        axs[0].legend(loc='upper right')
        axs[0].xaxis.set_major_locator(ticker.MultipleLocator(200))
        axs[0].xaxis.set_minor_locator(ticker.MultipleLocator(50))
        axs[0].yaxis.set_major_locator(ticker.MultipleLocator(0.25))
        axs[0].yaxis.set_minor_locator(ticker.MultipleLocator(0.15))
        axs[0].grid()

        axs[1].plot(x_lin, acc_list, 'red')
        axs[1].set(ylabel='discriminator accuracy (%)', xlabel='epoch')
        axs[1].xaxis.set_major_locator(ticker.MultipleLocator(200))
        axs[1].xaxis.set_minor_locator(ticker.MultipleLocator(50))
        axs[1].yaxis.set_major_locator(ticker.MultipleLocator(25))
        axs[1].yaxis.set_minor_locator(ticker.MultipleLocator(10))
        axs[1].grid()
        fig.tight_layout()
        fig.savefig('acc_and_loss.png')
        
        #self.GenPredict(5)

    def save_imgs(self, epoch, save_images=False):
        r, c = 3, 3  # Параметры вывода: r - количество строк, c - количество столбцов
        #noise = np.random.normal(0, 1, (r*c, self.img_shape[0], self.img_shape[1], self.img_shape[2]))  
        noise = np.random.normal(0, 1, (r*c, self.noise_shape[0], self.noise_shape[1], self.noise_shape[2]))
        gen_imgs = self.generator.predict(noise)  # Формируем генератором картинку на основании случайного входного сигнала
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5  # Выполняем обратное преобразование полученных значений изображения в диапазон от 0 до 1
        # Функция активации tanh, поэтому значения на выходе генератора лежат в диапазоне от -1 до + 1.
        fig, axs = plt.subplots(r, c, figsize=(5, 5))  # Создаем окно вывода r на c (5 на 5) ячеек
        cnt = 0  # Порядковый номер картинки
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :,0], cmap='gray')  # Записываем в axs[i,j] сгенерироввнное изображение из gen_img с индексом cnt
                axs[i, j].axis('off')  # Отключаем печать названия осей
                cnt += 1  # Увеличиваем индекс изображения
        # plt.show()                                                              # Выводим сгенерированные изображения
        if (save_images == True):
            fig.savefig(savedImgDir + '/gan_generated_image_epoch_%d.png' % epoch)  # Сохраняем изображения
        plt.close()

    def create_tendention_plot(self, save_interval, epoch_count):
        r = 1 + epoch_count // save_interval
        c = 1 + epoch_count % save_interval  # Параметры вывода: r - количество строк, c - количество столбцов
        print(r, c)
        fig, axs = plt.subplots(r, c)
        return fig, axs, r, c

    def save_tendention(self, fig, axs, all_colomns, cur_colomn):
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        gen_img = self.generator.predict(noise)
        gen_img = 0.5 * gen_img + 0.5

        curY, curX = cur_colomn // all_colomns, cur_colomn % all_colomns

        axs[curY, curX].imshow(gen_img,
                               cmap='gray')  # Записываем в axs[i,j] сгенерироввнное изображение из gen_img с индексом cnt
        axs[curY, curX].axis('off')  # Отключаем печать названия осей
        # Увеличиваем индекс изображения
        # plt.show()                                                              # Выводим сгенерированные изображения

    def close_tendention(self, fig):
        fig.savefig('tendence.png')  # Сохраняем изображения
        plt.close()

    def save(self, name):
        self.generator.save(name + '/gen')
        self.discriminator.save(name + '/dis')
        self.combined.save(name+'/comb')


    def GenPredict(self, count):
        #noise = np.random.normal(0, 1, (1, self.img_shape[0], self.img_shape[1], self.img_shape[2]))
        noise = np.random.normal(0, 1, (1, self.noise_shape[0], self.noise_shape[1], self.noise_shape[2]))
        for i in range(count):
            gen_img = self.generator.predict(noise)
            fig = plt.figure()
            plt.imshow(gen_img[0,:,:,0], cmap='gray')
            plt.savefig(generateMapTo + "/predictedImg_" + str(i) + ".png")


def trainGan(gan):
    gan.train(epochs=15000, batch_size=32, save_interval=30, save_images=True)
    #gan.save('gan_weights')
    gan.GenPredict(5)


def loadGanAndGen(gan):
    #loaded_model = tf.keras.models.load_model('gan_weights/gen')
    gan.checkpoint.restore(tf.train.latest_checkpoint(checkpointDir))
    gan.GenPredict(3)


def loadGanAndTrain(gan):
    #loaded_model = tf.keras.models.load_model('gan_weights/gen')
    gan.checkpoint.restore(tf.train.latest_checkpoint(checkpointDir))
    gan.train(epochs=25000, batch_size=32, save_interval=100, save_images=True)
    gan.GenPredict(5)



Запустим обучение

In [ ]:
gan = GAN()
trainGan(gan)
#loadGanAndGen(gan)
#loadGanAndTrain(gan)

Streaming output truncated to the last 5000 lines.
2015 [D loss: 0.710651, acc.: 43.23%] [G loss: 0.548956]
2016 [D loss: 0.705027, acc.: 41.16%] [G loss: 0.546568]
2017 [D loss: 0.706150, acc.: 48.02%] [G loss: 0.530590]
2018 [D loss: 0.704707, acc.: 45.10%] [G loss: 0.528455]
2019 [D loss: 0.705128, acc.: 43.18%] [G loss: 0.536728]
2020 [D loss: 0.704798, acc.: 41.80%] [G loss: 0.538867]
2021 [D loss: 0.704832, acc.: 44.01%] [G loss: 0.532364]
2022 [D loss: 0.704796, acc.: 45.97%] [G loss: 0.526679]
2023 [D loss: 0.704646, acc.: 42.18%] [G loss: 0.528495]
2024 [D loss: 0.704690, acc.: 45.32%] [G loss: 0.539809]
2025 [D loss: 0.704716, acc.: 49.96%] [G loss: 0.549413]
2026 [D loss: 0.704673, acc.: 52.04%] [G loss: 0.548633]
2027 [D loss: 0.704648, acc.: 55.94%] [G loss: 0.540868]
2028 [D loss: 0.704651, acc.: 40.52%] [G loss: 0.537613]
2029 [D loss: 0.704632, acc.: 47.85%] [G loss: 0.540755]
2030 [D loss: 0.704637, acc.: 46.96%] [G loss: 0.544043]
2031 [D loss: 0.704618, acc.: 46.15%]

KeyboardInterrupt: ignored